In [1]:
import pandas as pd

In [2]:
%load_ext lab_black

In [3]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
alt.renderers.set_embed_options(
    padding={"left": -10, "right": 0, "bottom": 0, "top": 0}
)

RendererRegistry.enable('default')

In [5]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [6]:
from df2gspread import df2gspread as d2g

In [7]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature="states")

In [10]:
wapo_shootings = pd.read_csv(
    "data/raw/waposhooting2.csv",
    thousands=",",
    dtype={"year": str},
    parse_dates=["date", "time"],
)

In [11]:
wapo_shootings.dtypes

uid                                               int64
nces_school_id                                   object
school_name                                      object
nces_district_id                                float64
district_name                                    object
date                                datetime64[ns, UTC]
school_year                                      object
year                                             object
time                                     datetime64[ns]
day_of_week                                      object
city                                             object
state                                            object
school_type                                      object
enrollment                                        int64
killed                                            int64
injured                                           int64
casualties                                        int64
shooting_type                                   

In [12]:
wapo_shootings["direct_victims"] = (
    wapo_shootings["enrollment"] + wapo_shootings["staffing"]
)

In [13]:
# wapo_shootings['year']=pd.to_datetime(wapo_shootings['year'],format="%Y")
# wapo_shootings['date']=pd.to_datetime(wapo_shootings['date'],format="%Y-%m-%d")

In [14]:
wapo_shootings.head()

,uid,nces_school_id,school_name,nces_district_id,district_name,date,school_year,year,time,day_of_week,city,state,school_type,enrollment,killed,injured,casualties,shooting_type,age_shooter1,gender_shooter1,race_ethnicity_shooter1,shooter_relationship1,shooter_deceased1,deceased_notes1,age_shooter2,gender_shooter2,race_ethnicity_shooter2,shooter_relationship2,shooter_deceased2,deceased_notes2,white,black,hispanic,asian,american_indian_alaska_native,hawaiian_native_pacific_islander,two_or_more,resource_officer,weapon,weapon_source,lat,long,staffing,low_grade,high_grade,lunch,county,state_fips,county_fips,ulocale,direct_victims
0,1,80480000707,Columbine High School,804800.0,Jefferson County R-1,1999-04-20 00:00:00+00:00,1998-1999,1999,2022-06-06 11:19:00,Tuesday,Littleton,Colorado,public,1965,13,21,34,indiscriminate,18.0,m,w,student,1.0,suicide,17.0,m,w,student,1.0,suicide,1783.0,16.0,112.0,42.0,12.0,NaN,NaN,1,12-gauge Savage-Springfield 67H pump-action sh...,purchased from friends,39.603910,-105.075000,89.6,9,12,41.0,Jefferson County,8,8059,21.0,2054.6
1,2,2.20E+11,Scotlandville Middle School,2200540.0,East Baton Rouge Parish School Board,1999-04-22 00:00:00+00:00,1998-1999,1999,2022-06-06 12:30:00,Thursday,Baton Rouge,Louisiana,public,588,0,1,1,targeted,14.0,m,NaN,former student (expelled),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,583.0,0.0,0.0,0.0,NaN,NaN,0,.22 caliber pistol,NaN,30.529958,-91.169966,39.0,6,8,495.0,East Baton Rouge Parish,22,22033,12.0,627.0
2,3,1.30E+11,Heritage High School,1304410.0,Rockdale County,1999-05-20 00:00:00+00:00,1998-1999,1999,2022-06-06 08:03:00,Thursday,Conyers,Georgia,public,1369,0,6,6,indiscriminate,15.0,m,w,student,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1189.0,136.0,28.0,15.0,1.0,NaN,NaN,1,".22 caliber rifle, 357-caliber Magnum handgun",NaN,33.626922,-84.047960,84.0,9,12,125.0,Rockdale County,13,13247,21.0,1453.0
3,4,4.22E+11,John Bartram High School,4218990.0,Philadelphia City SD,1999-10-04 00:00:00+00:00,1999-2000,1999,2022-06-06 10:00:00,Monday,Philadelphia,Pennsylvania,public,3147,0,1,1,targeted,17.0,m,NaN,student,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0,2736.0,27.0,170.0,5.0,NaN,NaN,1,RG .25-caliber pistol,purchased from friend,39.921509,-75.234108,41.0,9,12,2007.0,Philadelphia County,42,42101,11.0,3188.0
4,5,2.50E+11,Dorchester High School,2502790.0,Boston,1999-11-03 00:00:00+00:00,1999-2000,1999,2022-06-06 07:40:00,Wednesday,Boston,Massachusetts,public,1116,0,1,1,targeted,NaN,m,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,755.0,287.0,29.0,5.0,NaN,NaN,0,NaN,NaN,42.285268,-71.075901,NaN,9,12,543.0,Suffolk County,25,25025,11.0,NaN


In [15]:
grouped_victims = (
    wapo_shootings.groupby("year")
    .agg({"enrollment": "sum", "casualties": "sum"})
    .reset_index()
)

In [16]:
alt.Chart(grouped_victims).mark_point().encode(
    x=alt.X("year"), y=alt.Y("casualties"), size=alt.Size("enrollment")
)

alt.Chart(...)

In [17]:
grouped_victims["y"] = 2

alt.Chart(grouped_victims).mark_bar().encode(
    x=alt.X("year", stack="normalize"),
    y=alt.Y("y"),
    color=alt.Color(
        ("casualties"),
        scale=alt.Scale(scheme="blues", reverse=False, domain=(0, 45000)),
    ),
).properties(width=800, height=100)

alt.Chart(...)

In [18]:
alt.Chart(grouped_victims).mark_bar().encode(
    x=alt.X("year", stack="normalize"),
    y=alt.Y("y"),
    color=alt.Color(
        ("enrollment"),
        scale=alt.Scale(scheme="blues", reverse=False, domain=(0, 45000)),
    ),
).properties(width=800, height=100)

alt.Chart(...)

In [19]:
grouped_victims.dtypes

year          object
enrollment     int64
casualties     int64
y              int64
dtype: object

In [20]:
alt.Chart(grouped_victims).mark_trail().encode(
    x="year", y="enrollment", size="casualties"
)

alt.Chart(...)

In [21]:
## by locatino
locations = (
    wapo_shootings.groupby(["long", "lat", "year"])
    .agg({"enrollment": "sum", "casualties": "sum"})
    .reset_index()
)
locations.head()

,long,lat,year,enrollment,casualties
0,-157.837200,21.310400,2014,1416,1
1,-149.881113,61.213281,2018,429,1
2,-123.347120,43.208000,2006,2052,1
3,-123.166700,44.078800,2019,336,0
4,-122.832410,47.051956,2015,1477,0


In [22]:
background = (
    alt.Chart(states)
    .mark_geoshape(fill="white", stroke="gray")
    .properties(width=300, height=300)
    .project("albersUsa")
)

In [23]:
background

alt.Chart(...)

In [24]:
bubbles = (
    alt.Chart()
    .mark_circle(color="lightgray")
    .encode(longitude="long", latitude="lat", size=alt.Size("enrollment"))
    .project(type="albersUsa")
    .properties(width=300, height=300)
)

In [25]:
casualties = (
    alt.Chart()
    .mark_circle(color="teal")
    .encode(longitude="long", latitude="lat", size=alt.Size("casualties"))
    .project(type="albersUsa")
    .properties(width=300, height=300)
)

In [26]:
alt.layer(
    (background + bubbles + casualties), data=locations[locations["year"] > "2016"]
).facet("year:O", columns=3).configure_legend(orient="top")

alt.FacetChart(...)

In [27]:
import altair as alt
from vega_datasets import data

df = data.la_riots()

n = alt.topo_feature(
    "https://gist.githubusercontent.com/irisslee/70039051188dac8f64e14182b5a459a9/raw/2412c45551cff577f7b10604ca523bd3f4dd31d3/countytopo.json",
    "county",
)

LAbasemap = (
    alt.Chart(n)
    .mark_geoshape(fill="lightgray", stroke="white")
    .properties(width=400, height=400)
    .project("mercator")
)

points = (
    alt.Chart()
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.value(15),
        color="gender:N",
    )
)

alt.layer(LAbasemap, points, data=df).facet("gender:N")

alt.FacetChart(...)

In [28]:
states = alt.topo_feature(data.us_10m.url, feature="states")

In [29]:
## look at ratios
wapo_shootings["ratio"] = wapo_shootings["enrollment"] / wapo_shootings["killed"]

In [30]:
wapo_shootings.groupby("year").agg(
    {"enrollment": "sum", "casualties": "sum", "ratio": "sum"}
).reset_index()

,year,enrollment,casualties,ratio
0,1999,9345,49,NaN
1,2000,10093,7,NaN
2,2001,21379,27,NaN
3,2002,8110,8,NaN
4,2003,14028,13,NaN
5,2004,8338,12,NaN
6,2005,10751,26,NaN
7,2006,18215,25,NaN
8,2007,11023,18,NaN
9,2008,9969,10,NaN


In [31]:
# wapo_shootings['y'] = 2
# bydate=wapo_shootings.groupby('date').agg({'enrollment': "sum", "casualties": "sum"}).reset_index()

In [32]:
import datetime as dt

In [33]:
wapo_shootings["month"] = wapo_shootings["date"].dt.month
wapo_shootings["monthname"] = wapo_shootings["date"].dt.month_name()

In [34]:
monthly_shootings = (
    wapo_shootings.groupby(["month", "year", "monthname"])
    .agg({"enrollment": "sum", "casualties": "sum"})
    .reset_index()
)

In [35]:
monthly_shootings.head()

,month,year,monthname,enrollment,casualties
0,1,2000,January,2586,1
1,1,2001,January,5000,1
2,1,2002,January,2258,2
3,1,2003,January,1641,0
4,1,2005,January,1012,0


In [36]:
enrollment_grid = (
    alt.Chart(monthly_shootings)
    .mark_circle(color="#e8e8e8", opacity=0.5)
    .encode(
        x=alt.X("year", axis=alt.Axis(values=["2000", "2005", "2010", "2015", "2020"])),
        y=alt.Y(
            "monthname",
            sort=["January", "February"],
        ),
        size=alt.Size("enrollment", scale=alt.Scale(range=[1, 13323])),
    )
)

In [37]:
victims_grid = (
    alt.Chart(monthly_shootings)
    .mark_circle()
    .encode(
        x=alt.X(
            "year",
            title="",
            axis=alt.Axis(
                values=["2000", "2005", "2010", "2015", "2020"], ticks=False, offset=10
            ),
        ),
        y=alt.Y(
            "monthname",
            sort=["January", "February"],
            title="",
            axis=alt.Axis(grid=False, ticks=False, offset=20),
        ),
        size=alt.Size(
            "casualties",
            legend=None,
        ),
    )
)

In [38]:
(enrollment_grid + victims_grid).properties(
    width=500, height=500
).configure_legend().configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [39]:
wapo_shootings[["enrollment", "staffing", "killed", "injured", "casualties"]].sum()

enrollment    363705.000
staffing       19128.985
killed           185.000
injured          373.000
casualties       556.000
dtype: float64

In [40]:
(363705 + 19128) / 556

688.5485611510792

In [41]:
annual = (
    wapo_shootings.groupby(["year"])
    .agg({"enrollment": "sum", "casualties": "sum"})
    .reset_index()
)

In [42]:
annual["y"] = 0
annual["x"] = 0

In [43]:
casualty_year_dots = (
    alt.Chart()
    .mark_circle(color="black")
    .encode(
        x=alt.X("x", title=""),
        y=alt.Y("y", title=""),
        size=alt.Size("casualties", legend=None, scale=alt.Scale(range=[2, 95])),
    )
)

In [44]:
enrollment_year_dots = (
    alt.Chart()
    .mark_circle(color="#00eeef", opacity=0.3)
    .encode(
        x=alt.X("x", axis=alt.Axis(labels=False, ticks=False, grid=False)),
        y=alt.Y("y", axis=alt.Axis(labels=False, ticks=False, grid=False)),
        size=alt.Size("enrollment", legend=None, scale=alt.Scale(range=[1, 43971])),
    )
)

In [45]:
alt.layer(enrollment_year_dots, casualty_year_dots, data=annual).properties(
    width=50, height=30
).facet("year:O", columns=4)

alt.FacetChart(...)

In [49]:
wapo_shootings.groupby(["year"]).agg(
    {"enrollment": "sum", "casualties": "sum", "killed": "sum", "injured": "sum"}
).reset_index()

,year,enrollment,casualties,killed,injured
0,1999,9345,49,14,35
1,2000,10093,7,4,3
2,2001,21379,27,5,22
3,2002,8110,8,1,7
4,2003,14028,13,5,8
5,2004,8338,12,3,9
6,2005,10751,26,9,17
7,2006,18215,25,9,16
8,2007,11023,18,1,17
9,2008,9969,10,5,5
